<a href="https://colab.research.google.com/github/kardwalker/Building_Subword_Tokenizer/blob/main/Tokenizer_library.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training a new tokenizer from old one


In [ ]:
!pip install -q transformers
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from datasets import load_dataset
raw_dataset = load_dataset("code_search_net" , "python" )
raw_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


code_search_net.py:   0%|          | 0.00/8.44k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/12.9k [00:00<?, ?B/s]

The repository for code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code_search_net.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


python.zip:   0%|          | 0.00/941M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 412178
    })
    test: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 22176
    })
    validation: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 23107
    })
})

In [ ]:
raw_dataset["train"]

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 412178
})

whole_func_string column to train our tokenizer

In [ ]:
print(raw_dataset["train"]["whole_func_string"])

Buffered data was truncated after reaching the output size limit.

In [ ]:
def handle_simple_response(self, timeout_ms = None , info_cb = None):
  return self._accept_response("OKAY" , info_cb  , timeout_ms = timeout_ms)

In [ ]:
training_corpus = (
    raw_dataset["train"][i  : i + 1000]["whole_func_string"]
    for i in range(0 , len(raw_dataset["train"]) , 1000)
)

In [ ]:
gen = (i for i in range(10))
print(list(gen))
print(list(gen))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[]


In [ ]:
def get_training_corpus():
  return (
      raw_dataset["train"][i : i+ 1000]["whole_func_string"]
      for i in range(0 ,len(raw_dataset["train"]) ,1000)
  )

training_corpus = get_training_corpus()

In [ ]:
def get_training_corpus():
  dataset = raw_dataset["train"]
  for start_idx in range(0 ,len(dataset) , 1000):
    samples = dataset[start_idx : start_idx + 1000]
    yield samples["whole_func_string"]


Training new tokenizer

In [ ]:
from transformers  import AutoTokenizer
old_tokenizer = AutoTokenizer.from_pretrained("gpt2")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
ex = """def add_numbers(a , b):
  ''' Add the two numbers 'a' and 'b'.'''
  return a + b """

tokens = old_tokenizer.tokenize(ex)
print(tokens)
# This tokenizer has a few special symbols, like Ġ and Ċ, which denote spaces and newlines, respectively


['def', 'Ġadd', '_', 'n', 'umbers', '(', 'a', 'Ġ,', 'Ġb', '):', 'Ċ', 'Ġ', "Ġ'", "''", 'ĠAdd', 'Ġthe', 'Ġtwo', 'Ġnumbers', "Ġ'", 'a', "'", 'Ġand', "Ġ'", 'b', "'", ".'", "''", 'Ċ', 'Ġ', 'Ġreturn', 'Ġa', 'Ġ+', 'Ġb', 'Ġ']


Train a new tokenizer using the method
train_new_from_iterator()

In [ ]:
trained_tokenizer = old_tokenizer.train_new_from_iterator(training_corpus , 52000)
# Note that AutoTokenizer.train_new_from_iterator() only works if the tokenizer you are using is a “fast” tokenizer.

# Embedding

MTEB( Massive text embedding Benchmark) is a massive benchmark for measuring the performance of text embedding models on diverse embedding tasks.

In [ ]:
"""Text Embedding are vector representation of text that encode semantic information
As machine require numerical inputs to perform computations ,text embedding are a crucial
component of many downstream NLP applicatons."""

In [ ]:
!pip install mteb

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from mteb import MTEB
from sentence_transformers import SentenceTransformer

model_name = "average_word_embeddings_komninos"
model = SentenceTransformer(model_name)

evaluation = MTEB(tasks=["Banking77Classification"])
results = evaluation.run(model, output_folder=f"results/{model_name}")


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# Inside the token-classifaction pipeline

In [ ]:
from transformers import pipeline
token_classifier = pipeline(
    "token-classification" , model = "dbmdz/bert-large-cased-finetuned-conll03-english"
)
token_classifier("myself Aman Yadav , i study at IIT(ISM)")

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity': 'I-PER',
  'score': 0.9992618,
  'index': 2,
  'word': 'Am',
  'start': 7,
  'end': 9},
 {'entity': 'I-PER',
  'score': 0.99801517,
  'index': 3,
  'word': '##an',
  'start': 9,
  'end': 11},
 {'entity': 'I-PER',
  'score': 0.99764496,
  'index': 4,
  'word': 'Ya',
  'start': 12,
  'end': 14},
 {'entity': 'I-PER',
  'score': 0.5426543,
  'index': 5,
  'word': '##da',
  'start': 14,
  'end': 16},
 {'entity': 'I-PER',
  'score': 0.9950836,
  'index': 6,
  'word': '##v',
  'start': 16,
  'end': 17},
 {'entity': 'I-ORG',
  'score': 0.9438734,
  'index': 11,
  'word': 'II',
  'start': 31,
  'end': 33},
 {'entity': 'I-ORG',
  'score': 0.713781,
  'index': 12,
  'word': '##T',
  'start': 33,
  'end': 34},
 {'entity': 'I-ORG',
  'score': 0.8193158,
  'index': 14,
  'word': 'IS',
  'start': 35,
  'end': 37},
 {'entity': 'I-ORG',
  'score': 0.74232143,
  'index': 15,
  'word': '##M',
  'start': 37,
  'end': 38}]

In [ ]:
from transformers import pipeline
token_classifier = pipeline(
    "token-classification" , aggregation_strategy ="simple"
)
token_classifier("Myself Aman , I love to code in python and rust")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity_group': 'PER',
  'score': 0.83661747,
  'word': 'Aman',
  'start': 7,
  'end': 11}]

without using the pipeline

In [ ]:
from transformers import AutoTokenizer , AutoModelForTokenClassification
checkpoint = "dbmdz/bert-large-cased-finetuned-conll03-english"
model = AutoModelForTokenClassification.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
ex = "My name is Aman Yadav , I study at IIT(ISM)"
inputs = tokenizer(ex , return_tensors = "pt")
outputs = model(**inputs)

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
print(inputs["input_ids"].shape)
print(outputs.logits.shape)

torch.Size([1, 20])
torch.Size([1, 20, 9])


In [ ]:
import torch
prob = torch.nn.functional.softmax(outputs.logits , dim = -1)[0].tolist()
predictions = outputs.logits.argmax(dim = -1)[0].tolist()
print(predictions)

[0, 0, 0, 0, 4, 4, 4, 4, 4, 0, 0, 0, 0, 6, 6, 0, 6, 6, 0, 0]


In [ ]:
model.config.id2label

{0: 'O',
 1: 'B-MISC',
 2: 'I-MISC',
 3: 'B-PER',
 4: 'I-PER',
 5: 'B-ORG',
 6: 'I-ORG',
 7: 'B-LOC',
 8: 'I-LOC'}

# Normalization And Pre-Tokenization

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
type(tokenizer.backend_tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizers.Tokenizer

Normalizer attribute of tokenizer object is normalize_str()

In [ ]:
text = "love fades in times"
"""text_normalized = tokenizer.backend_tokenizer.normalize_str(text)
AttributeError: 'tokenizers.Tokenizer' object has no attribute 'normalize_str'
"""
tokenizer.convert_ids_to_tokens(tokenizer.encode(text))

['[CLS]', 'love', 'fades', 'in', 'times', '[SEP]']

Pre-tokenization

In [ ]:
tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str("It's been a long day")
gpt_tokenizer = AutoTokenizer.from_pretrained("gpt2")
gpt_tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str("It's been a long day")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

[('It', (0, 2)),
 ("'s", (2, 4)),
 ('Ġbeen', (4, 9)),
 ('Ġa', (9, 11)),
 ('Ġlong', (11, 16)),
 ('Ġday', (16, 20))]

In [ ]:
t5_tokenizer = AutoTokenizer.from_pretrained("t5-small") # based on SentencePiece Algorithm
t5_tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str("It's been a long day")

fnet_tokenizer = AutoTokenizer.from_pretrained("google/fnet-base")
fnet_tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str("It's been a long day")

Retribert_tokenizer = AutoTokenizer.from_pretrained("yjernite/retribert-base-uncased")
Retribert_tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str("It's been a long day")

tokenizer_config.json:   0%|          | 0.00/455 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/708k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/487 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

[('It', (0, 2)),
 ("'", (2, 3)),
 ('s', (3, 4)),
 ('been', (5, 9)),
 ('a', (10, 11)),
 ('long', (12, 16)),
 ('day', (17, 20))]

SentencePiece tokenization algorithm



In [ ]:
# main feature of SentencePiece is reversible tokenization


# Byte-Pair Encoding tokenization

In [ ]:
# Byte-Pair Encoding(BPE)
# byte-level BPE

"""
Tokenization Algorithm

Normalization
Pre-tokenization
Splitting the words into individual charaters
Applying the merge rules learned in order to those splits

"""

In [ ]:
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
for text in corpus:
  print(text)

tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)

This is the Hugging Face Course.
This chapter is about tokenization.
This section shows several tokenizer algorithms.
Hopefully, you will be able to understand how they are trained and generate tokens.


[('Hopefully', (0, 9)),
 (',', (9, 10)),
 ('Ġyou', (10, 14)),
 ('Ġwill', (14, 19)),
 ('Ġbe', (19, 22)),
 ('Ġable', (22, 27)),
 ('Ġto', (27, 30)),
 ('Ġunderstand', (30, 41)),
 ('Ġhow', (41, 45)),
 ('Ġthey', (45, 50)),
 ('Ġare', (50, 54)),
 ('Ġtrained', (54, 62)),
 ('Ġand', (62, 66)),
 ('Ġgenerate', (66, 75)),
 ('Ġtokens', (75, 82)),
 ('.', (82, 83))]

In [ ]:
from collections import defaultdict
word_freqs = defaultdict(int)
import pdb
for text in corpus:

  words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
  new_words = [word for word , offset in words_with_offsets]

  for word in new_words:
    word_freqs[word] += 1


print(word_freqs)
print(type(word_freqs))


defaultdict(<class 'int'>, {'This': 3, 'Ġis': 2, 'Ġthe': 1, 'ĠHugging': 1, 'ĠFace': 1, 'ĠCourse': 1, '.': 4, 'Ġchapter': 1, 'Ġabout': 1, 'Ġtokenization': 1, 'Ġsection': 1, 'Ġshows': 1, 'Ġseveral': 1, 'Ġtokenizer': 1, 'Ġalgorithms': 1, 'Hopefully': 1, ',': 1, 'Ġyou': 1, 'Ġwill': 1, 'Ġbe': 1, 'Ġable': 1, 'Ġto': 1, 'Ġunderstand': 1, 'Ġhow': 1, 'Ġthey': 1, 'Ġare': 1, 'Ġtrained': 1, 'Ġand': 1, 'Ġgenerate': 1, 'Ġtokens': 1})
<class 'collections.defaultdict'>


NameError: name 'offset' is not defined

Efficiency: If the corpus is large, you might consider optimizing memory usage, possibly by using a Counter instead of defaultdict(int), which is more concise and efficient for counting.

In [ ]:
from collections import Counter
word_freqs = Counter()

for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    word_freqs.update(new_words)

print(word_freqs)
print(type(word_freqs))

Counter({'.': 4, 'This': 3, 'Ġis': 2, 'Ġthe': 1, 'ĠHugging': 1, 'ĠFace': 1, 'ĠCourse': 1, 'Ġchapter': 1, 'Ġabout': 1, 'Ġtokenization': 1, 'Ġsection': 1, 'Ġshows': 1, 'Ġseveral': 1, 'Ġtokenizer': 1, 'Ġalgorithms': 1, 'Hopefully': 1, ',': 1, 'Ġyou': 1, 'Ġwill': 1, 'Ġbe': 1, 'Ġable': 1, 'Ġto': 1, 'Ġunderstand': 1, 'Ġhow': 1, 'Ġthey': 1, 'Ġare': 1, 'Ġtrained': 1, 'Ġand': 1, 'Ġgenerate': 1, 'Ġtokens': 1})
<class 'collections.Counter'>


computing the base vocalburay

In [ ]:
print(word_freqs.keys())

dict_keys(['This', 'Ġis', 'Ġthe', 'ĠHugging', 'ĠFace', 'ĠCourse', '.', 'Ġchapter', 'Ġabout', 'Ġtokenization', 'Ġsection', 'Ġshows', 'Ġseveral', 'Ġtokenizer', 'Ġalgorithms', 'Hopefully', ',', 'Ġyou', 'Ġwill', 'Ġbe', 'Ġable', 'Ġto', 'Ġunderstand', 'Ġhow', 'Ġthey', 'Ġare', 'Ġtrained', 'Ġand', 'Ġgenerate', 'Ġtokens'])


In [ ]:
alphabet = []
for word in word_freqs.keys():
  for letter in word:
    if letter not in alphabet:
      alphabet.append(letter)

alphabet.sort()
print(alphabet)


[',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', 'Ġ']


In [ ]:
vocab = ["<|endoftext|>"] + alphabet.copy()

function that computes the frequency of each pair

In [ ]:
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

In [ ]:
splits = { word : [c for c in word] for word in word_freqs.keys()}

In [ ]:
pair_freqs = compute_pair_freqs(splits)

for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

('T', 'h'): 3
('h', 'i'): 3
('i', 's'): 5
('Ġ', 'i'): 2
('Ġ', 't'): 7
('t', 'h'): 3


In [ ]:
cbest_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

('Ġ', 't') 7


# WordPiece tokenization

google never open-sourced its implementation of the trianing algo of WordPiece , its just the best guess based on the published literature

In [ ]:
# score = (freq_of_pair)/(freq_of_first_element * freq_of_second_element)
"""
WordPiece finds the longest subword that is in vocabulary

"""

Implementing WordPiece algo

In [ ]:
corpus = ["This is the hugging Face Course.",
          "This chapter is about tokenization.",
          "This section shows several tokenizer algorithms.",
          "Hopefully , you will br able to understand how they are trained and generate tokens"]

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:

word_freqs = defaultdict(int)
for text in corpus:
  words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
  new_words = [word for word , offset in words_with_offsets]
  for word in new_words:
    word_freqs[word] += 1

print(word_freqs)

defaultdict(<class 'int'>, {'This': 3, 'is': 2, 'the': 1, 'hugging': 1, 'Face': 1, 'Course': 1, '.': 3, 'chapter': 1, 'about': 1, 'tokenization': 1, 'section': 1, 'shows': 1, 'several': 1, 'tokenizer': 1, 'algorithms': 1, 'Hopefully': 1, ',': 1, 'you': 1, 'will': 1, 'br': 1, 'able': 1, 'to': 1, 'understand': 1, 'how': 1, 'they': 1, 'are': 1, 'trained': 1, 'and': 1, 'generate': 1, 'tokens': 1})


In [ ]:
alphabet = []
for word in word_freqs.keys():
  if word[0] not in alphabet:
    alphabet.append(word[0])
    for letter in word[1:]:
      if f"##{letter}" not in alphabet:
        alphabet.append(f"##{letter}")

alphabet.sort()
print(alphabet)


['##a', '##b', '##c', '##d', '##e', '##f', '##g', '##h', '##i', '##l', '##n', '##o', '##p', '##r', '##s', '##t', '##u', '##y', ',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'g', 'h', 'i', 's', 't', 'u', 'w', 'y']


In [ ]:

vocab = ["[UNK]"] + ["[CLS]"] + ["[SEP]"] + ["[PAD]"] + ["[MASK]"] + alphabet.copy()

In [ ]:
splits = {
    word : [c if i == 0 else f"##{c}" for i, c in enumerate(word)]
    for word in word_freqs.keys()
}

In [ ]:
def compute_pair_freqs(splits):
  letter_freqs = defaultdict(int)
  pair_freqs = defaultdict(int)
  for word, freq in word_freqs.items():
    split = splits[word]
    if len(split) == 1:
      letter_freqs[split[0]] += freq
      continue
    for i in range(len(split) - 1):
      pair = (split[i], split[i + 1])
      letter_freqs[split[i]] += freq
      pair_freqs[pair] += freq
    letter_freqs[split[-1]] += freq

  scores = {
    pair : freq / (letter_freqs[pair[0]] * letter_freqs[pair[1]])
    for pair , freq in pair_freqs.items()
}
  return scores


In [ ]:
pair_scores = compute_pair_freqs(splits)
for i, key in enumerate(pair_freqs.keys()):
  print(f"{key} : {[pair_scores]}")
  if i >= 5:
    break

('T', 'h') : [{('T', '##h'): 0.125, ('##h', '##i'): 0.03409090909090909, ('##i', '##s'): 0.02727272727272727, ('i', '##s'): 0.1, ('t', '##h'): 0.03571428571428571, ('##h', '##e'): 0.0125, ('h', '##u'): 0.1, ('##u', '##g'): 0.05, ('##g', '##g'): 0.0625, ('##g', '##i'): 0.022727272727272728, ('##i', '##n'): 0.01652892561983471, ('##n', '##g'): 0.022727272727272728, ('F', '##a'): 0.14285714285714285, ('##a', '##c'): 0.07142857142857142, ('##c', '##e'): 0.025, ('C', '##o'): 0.07692307692307693, ('##o', '##u'): 0.046153846153846156, ('##u', '##r'): 0.02, ('##r', '##s'): 0.02, ('##s', '##e'): 0.005, ('c', '##h'): 0.125, ('##h', '##a'): 0.017857142857142856, ('##a', '##p'): 0.07142857142857142, ('##p', '##t'): 0.07142857142857142, ('##t', '##e'): 0.014285714285714285, ('##e', '##r'): 0.025, ('a', '##b'): 0.2, ('##b', '##o'): 0.038461538461538464, ('##u', '##t'): 0.02857142857142857, ('t', '##o'): 0.04395604395604396, ('##o', '##k'): 0.07692307692307693, ('##k', '##e'): 0.05, ('##e', '##n'): 0

In [ ]:
best_pair = ""
max_score = None
for pair , score in pair_scores.items():
  if max_score is None or max_score < score:
    best_pair = pair
    max_pair = score


print(best_pair , max_score)

('##n', '##s') None


# building a tokenizer

In [ ]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset("wikitext" ,name = "wikitext-2-raw-v1" , split = "train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
def get_training_corpus():
  for i in range(0 ,len(dataset) , 1000):
    yield dataset[i : i + 1000]["text"]

In [ ]:
with open("wikitext-2-raw-v1.txt" , "w" , encoding = "utf-8") as fp:
  for line in dataset:
    fp.write(line["text"] + "\n")

In [ ]:
from tokenizers import (decoders ,models, normalizers ,pre_tokenizers , processors ,trainers ,Tokenizer,)
tokenizer = Tokenizer(models.WordPiece(unk_token = "[UNK]")


SyntaxError: incomplete input (<ipython-input-6-9cf6c1d4a42e>, line 2)

# pandas


In [ ]:
data = { 'a':  [2 ,3 ,4],
        'b' : ['[1,2,3]' , '[4,5,6]', '[3,4, 2,9]']}

import ast        # asbtract syntax tree
import pandas as pd
df = pd.DataFrame(data)
print(df)
# converting the string representation of the list into actual list
df.b = df.b.apply(ast.literal_eval)
df.info()
type([i for i in df.b[0]])

   a           b
0  2     [1,2,3]
1  3     [4,5,6]
2  4  [3,4, 2,9]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   a       3 non-null      int64 
 1   b       3 non-null      object
dtypes: int64(1), object(1)
memory usage: 176.0+ bytes


list